$$
\begin{array}{r}
\min 2 x_{1}-2 x_{2}+2 x_{3} \\
\text { s.t. } x_{1} \geq 2, x_{2} \geq 1, x_{3} \geq 1 \\
2 x_{1}-x_{2}+2 x_{3} \leq 6 \\
x_{1}-4 x_{2}+2 x_{3} \leq-15 \\
-2 x_{1}+2 x_{2}+x_{3} \leq-8
\end{array}
$$

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 4.9 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
OP = ConcreteModel()

In [5]:
N = 3 #number of decision variables 
M = 3 #number of constraints without considering the bound constraints  
lb=np.array([2,1,1])
ub=np.array([np.inf,np.inf,np.inf]) 
obj_coef_c = np.array([2,-2,2]) 
constr_coef_A=np.array([[2,-1,2],[1,-4,2],[-2,2,1]])
constr_rhs_b=np.array([6,-15,-8])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[0 1 2]
[0 1 2]


In [6]:
OP.x=Var(col_indices)
OP.constraints = ConstraintList()

In [7]:
for i in col_indices:
  OP.x[i].setlb(lb[i])
  OP.x[i].setub(ub[i])


In [8]:
for i in row_indices:
  OP.constraints.add(sum(constr_coef_A[i][j]*OP.x[j] for j in col_indices) <= constr_rhs_b[i])

In [9]:
OP.objective = Objective(expr =summation(obj_coef_c,OP.x), sense=minimize)
OP.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [10]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [11]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
#.solve(model1).write()
result = opt.solve(OP)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.02797555923461914

Solver status: ok
Solver termination condition: other


In [12]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155879 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [13]:
opt_cbc = SolverFactory('cbc')

In [14]:
result = opt_cbc.solve(OP)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.04763197898864746

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.04763197898864746


$ \large{solution \ \ of \ \ 3.1 }$ $ \text{when we use glpk solver status is ok and termination condition is other} \\ \text{when we use cbc solver it is clearly mention solver status is warning and termination condition is infeasible}  \\ \text{in glpk solver we can not get conclusion from status and termination condition} \\  \text{ but in cbc solver we get meaningful termination condition :- infeasible i.e we can not get value of variable} \\   \text{that fullfill our constraints and objective function } $

In [15]:
OP.objective.set_sense(maximize)

In [16]:
OP.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [17]:
result = opt_cbc.solve(OP)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.029181957244873047

Solver status: warning
Solver termination condition: infeasible


in 1.2(a) \\
solver status is warning \\
and solver termination condition is infeasible \\
i.e there is no value of variable that fulfill our constraint and objective function

In [18]:
OP.objective.set_sense(minimize)

In [19]:
#reset the upper bound of x2 and x3
OP.x[1].setub(5)
OP.x[2].setub(10)

OP.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :     5 : False :  True :  Reals
          2 :     1 :  None :    10 : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [20]:
result = opt_cbc.solve(OP)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.023743391036987305

Solver status: warning
Solver termination condition: infeasible


$ \huge{solution \ \ of \ \ 1.2(b) } $ $ \text{solver status is warning and solver termination condition is infeasible i.e we can not get value of variable that fulfill our constraints and objective}$

In [21]:
OP.x[1].setub(np.inf)
OP.x[2].setub(np.inf)

In [22]:
new_constr_coef = [0,-1,1]
new_constr_rhs = 16
OP.constraints.add(sum(new_constr_coef[j]*OP.x[j] for j in col_indices) <= new_constr_rhs)
OP.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=4, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :  

In [23]:
result = opt_cbc.solve(OP)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: 11.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 0
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.01756739616394043

Solver status: warning
Solver termination condition: infeasible


$ \huge{solution \ \ of \ \ 1.2(c) }$ $ \text{solver status is warning and solver termination condition is infeasible i.e we can not get value of variable that fulfill our constraints and objective}$

In [24]:
#deactivate the old objective
OP.objective.deactivate()

In [25]:
OP.constraints[4].deactivate()

In [26]:
coef_new_objective = np.array([1,2,3])

In [27]:
OP.new_objective = Objective(expr = summation(coef_new_objective,OP.x),sense =minimize)

In [28]:
OP.constraints[1].deactivate()

In [29]:
coef_new_constraints =np.array([0,1,1])

In [30]:
OP.constraints.add( expr= summation(coef_new_constraints,OP.x) >=25)

In [31]:
OP.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + 2*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints

In [32]:
result = opt_cbc.solve(OP)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 79.5
  Upper bound: 79.5
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.02944660186767578
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [33]:
# display solution
print('\nObjective = ', OP.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', OP.x[i].value)
print('\nConstraints')
OP.constraints.display()


Objective =  79.5

Decision Variables
x[ 0 ] =  28.5
x[ 1 ] =  24.0
x[ 2 ] =  1.0

Constraints
constraints : Size=5
    Key : Lower : Body  : Upper
      2 :  None : -65.5 : -15.0
      3 :  None :  -8.0 :  -8.0
      5 :  25.0 :  25.0 :  None


$ \huge{solution \ \ of  \ \ 1.2(d)} \\ \ objective \ \ function \ \ value = 79.5 \\ value \ \  of \ \ decision \ \ variable \\ x_1 = 28.5 \\ x_2=24.0 \\ x_3=1.0 \\ constraints2 \ \ is \ \ inactive \\ constraint3 \ \ is \ \ active \\ constraint5  \ \ is  \ \ active $